In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib


In [8]:
# import pandas as pd
# from sklearn.feature_selection import RFE
# from sklearn.ensemble import RandomForestClassifier
# # 假设你的CSV文件名为'data.csv'
# # 并且目标变量的列名为'target'
# df = pd.read_csv('train_dataset.csv')

# # 分离特征和目标变量
# X = data.iloc[:, :-1]
# y = data.iloc[:, -1]

# # 使用RandomForestClassifier作为基模型
# model = RandomForestClassifier()

# # 使用RFE选择50个特征
# selector = RFE(model, n_features_to_select=50, step=1)
# selector = selector.fit(X, y)

# # 获取选中的特征
# selected_features = X.columns[selector.support_]

# # 打印选中的特征
# print("Selected features:")
# print(selected_features)

# # 创建一个包含选中特征的新数据框
# X_selected = X[selected_features]

# # 保存新数据框到新的CSV文件
# X_selected['target'] = y
# X_selected.to_csv('data-RFE.csv', index=False)

In [9]:
# # 预测集生成
# import pandas as pd

# # 加载保存的训练集RFE结果的CSV文件
# df_rfe_results = pd.read_csv('data-RFE.csv')

# # 获取选中的特征列表
# selected_features = df_rfe_results.columns[:-1]  # 假设最后一列是目标变量，所以去掉最后一列

# # 打印选中的特征
# print("Selected features:")
# print(selected_features)

# # 加载预测集数据
# df_predict = pd.read_csv('test_dataset.csv')

# # 保留预测集中与训练集相同的选中特征
# X_predict = df_predict[selected_features]

# # 保存预测集的特征到新的CSV文件
# X_predict.to_csv('test_dataset-RFE.csv', index=False)


In [10]:
# 读取数据
data = pd.read_csv('data-RFE.csv')


In [11]:
# 假设目标变量是最后一列，特征变量是其他列
X = data.iloc[:, :-1]
y = data.iloc[:, -1]


In [12]:
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


In [13]:
# # 绘制目标值分布直方图
# plt.figure(figsize=(14, 8), dpi=100)
# plt.hist(y, bins=30, color='plum', edgecolor='black', alpha=0.7)
# plt.xlabel('Target Value', fontsize=14)
# plt.ylabel('Frequency', fontsize=14)
# plt.title('Distribution of Target Values', fontsize=16)
# plt.grid(True)
# plt.savefig('Target_Value_Distribution.png', dpi=300)
# plt.show()
# plt.close()

In [16]:
!pip install --upgrade lightgbm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.7 MB/s eta 0:00:0000:0100:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lightgbm: filename=lightgbm-4.4.0-py3-none-linux_x86_64.whl size=3091329 sha256=c48e3d45dfc6121ba1d49587d6c2c882584a89ec17e6639f7436a9f4d6be8aa8
  Stored in directory: /home/customer/.cache/pip/wheels/61/69/b2/4cb581656980662b1585f36e62c09dc4bd6d9c1d400fc042fb
Successfully built lightgbm


In [19]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# 定义 LightGBM 的数据集
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# 设置 LightGBM 的参数
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# 使用回调参数进行训练并实现提前停止
num_round = 100  # 迭代次数
early_stopping_callback = lgb.early_stopping(stopping_rounds=10)
bst = lgb.train(params, train_data, num_boost_round=num_round, valid_sets=[test_data], callbacks=[early_stopping_callback])

# 在测试集上进行预测
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)
# 将预测结果转为二分类的结果（0或1）
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

# 评估模型性能
accuracy = accuracy_score(y_test, y_pred_binary)
conf_matrix = confusion_matrix(y_test, y_pred_binary)
class_report = classification_report(y_test, y_pred_binary)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 126, number of negative: 84
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001244 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2297
[LightGBM] [Info] Number of data points in the train set: 210, number of used features: 50
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.600000 -> initscore=0.405465
[LightGBM] [Info] Start training from score 0.405465
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

In [21]:
# 加载新数据集
new_data = pd.read_csv('test_dataset-RFE.csv')

# 假设 new_data 是包含特征的 DataFrame，需要与训练时的特征格式一致

# 使用已训练好的模型进行预测
y_pred_new = bst.predict(new_data, num_iteration=bst.best_iteration)

# 创建包含预测结果的 DataFrame
result_df = pd.DataFrame({'uuid': range(1, len(new_data) + 1), 'Label': y_pred_new})

# 将结果保存为 CSV 文件
result_df.to_csv('prediction_results.csv', index=False)

# 打印预测结果的前几行
print(result_df.head())

   uuid     Label
0     1  0.848725
1     2  0.838199
2     3  0.838199
3     4  0.838199
4     5  0.838199


In [22]:

# 使用已训练好的模型进行预测
y_pred_proba = bst.predict(new_data, num_iteration=bst.best_iteration)

# 设置阈值
threshold = 0.5  # 例如，设置0.5作为阈值

# 将概率转换为二分类结果（0或1）
y_pred_binary = [1 if pred > threshold else 0 for pred in y_pred_proba]

# 创建包含预测结果的 DataFrame
result_df = pd.DataFrame({'uuid': range(1, len(new_data) + 1), 'Label': y_pred_binary})

# 将结果保存为 CSV 文件
result_df.to_csv('prediction_results.csv', index=False)

# 打印预测结果的前几行
print(result_df.head())

   uuid  Label
0     1      1
1     2      1
2     3      1
3     4      1
4     5      1


In [24]:
# 加载模型
bst = lgb.Booster(model_file='lightgbm.txt')



[LightGBM] [Fatal] Could not open lightgbm.txt


LightGBMError: Could not open lightgbm.txt